## Caricamento/Download del modello

In [ ]:
# Per configurare la quantizzazione del modello, richiede `bitsandbytes`
# Esempio per il caricamento in 4bit
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

model_path = "" # Repository di Huggingface del modello se si vuole scaricarlo oppure path locale se è già stato salvato

# Può succedere che modello e tokenizer siano da scaricare separatamente, in genere sono insieme

tokenizer = transformers.AutoTokenizer.from_pretrained(model_path)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_path,
    #trust_remote_code=True,    # alcuni modelli lo richiedono
    quantization_config=bnb_config,
    device_map='auto',  # oppure per specificare la GPU n°0:  device_map = {"":0}
    torch_dtype=torch.float16,
)

## Pipeline per specificare le proprietà di funzionamento del modello

In [ ]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline

pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,     # per controllare la lunghezza dell'output
    temperature=0.0,
    top_p=0.95,
    repetition_penalty=1.15,    # default = 1
)

local_llm = HuggingFacePipeline(pipeline=pipe)

## Test

In [ ]:
response = local_llm("Tell me the name of the capital of England")
print(response)